# 数据预处理

数据预处理包含对图像进行数据增强和对标签进行处理等操作，这里主要介绍图像处理部分。

在模型训练过程中有时会遇到过拟合的问题，其中一个解决方法就是对训练数据做数据处理或增强。数据处理通过对数据进行处理得到不同的图像，从而增强模型的泛化性。





## paddle.vision.transforms 介绍

飞桨框架在 [paddle.vision.transforms](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/vision/Overview_cn.html#about-transforms) 下内置了数十种数据处理方法，可以通过以下代码查看：

In [5]:
import paddle
print('数据处理方法：', paddle.vision.transforms.__all__)

数据处理方法： ['BaseTransform', 'Compose', 'Resize', 'RandomResizedCrop', 'CenterCrop', 'RandomHorizontalFlip', 'RandomVerticalFlip', 'Transpose', 'Normalize', 'BrightnessTransform', 'SaturationTransform', 'ContrastTransform', 'HueTransform', 'ColorJitter', 'RandomCrop', 'Pad', 'RandomRotation', 'Grayscale', 'ToTensor', 'to_tensor', 'hflip', 'vflip', 'resize', 'pad', 'rotate', 'to_grayscale', 'crop', 'center_crop', 'adjust_brightness', 'adjust_contrast', 'adjust_hue', 'normalize']


对于飞桨框架内置的数据处理方法，可以单个初始化调用，也可以将多个数据处理方法进行组合使用，具体使用方式如下：

* 单个使用

In [14]:
from paddle.vision.transforms import Resize

# 定义想要使用的数据处理方式，这里初始化一个改变图片大小的变换
transform = Resize(size=28)

* 多个组合使用

这种使用模式下，需要先定义好每个数据处理方法，然后用`Compose` 进行组合

In [35]:
from paddle.vision.transforms import Compose, RandomRotation

# 定义想要使用的数据处理方式，这里包括随机旋转，改变图片大小
transform = Compose([RandomRotation(10), Resize(size=32)])

## 在数据集中定义数据预处理操作

定义好数据预处理方法后，可以直接在 DataSet 中使用，下面介绍介绍两种数据处理使用方式，一种是基于框架内置数据集，一种是基于自定义的数据集。

### 基于框架内置数据集

在框架内置数据集中使用内置的数据处理方法时，只需要将数据处理操方法传递给 `transform` 字段即可。

In [12]:
# 通过transform参数传递定义好的数据增强方法即可完成对自带数据集的增强
train_dataset = paddle.vision.datasets.MNIST(mode='train', transform=transform)

### 基于自定义的数据集

对于自定义的数据集，可以在数据集的 `__init__` 函数中定义数据处理方法，之后在 `__getitem__` 方法中对返回的数据进行应用, 如下述代码所示：


In [16]:
import numpy as np
from paddle.io import Dataset

class MyDataset(Dataset):    
    def __init__(self, num_samples, image_size=(28,28), class_num=10):
                super(MyDataset, self).__init__()
        self.num_samples = num_samples
        self.image_size = image_size
        self.class_num = class_num
        
        # 在 `__init__` 中定义数据处理方法，此处为随机旋转
        self.transform = RandomRotation(10)

    def __getitem__(self, index):        
        image = np.random.rand(*self.image_size)
        
        # 在 `__getitem__` 中对数据集使用数据处理方法
        data = self.transform(data)
        
        
        image = np.expand_dims(image, axis=0)
        label = np.random.randint(0, self.class_num - 1)

        return image, label

    def __len__(self):        
        return self.num_samples

## 数据处理的几种方法介绍

待补充几个典型方法和转换前后对比示例